# SK-7-MultiModal : Images, Audio et Vision

**Navigation** : [<< 06-ProcessFramework](06-SemanticKernel-ProcessFramework.ipynb) | [Index](README.md) | [08-MCP >>](08-SemanticKernel-MCP.ipynb)

---

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Generer des **images** avec DALL-E 3
2. Analyser des **images** avec GPT-4 Vision
3. Transcrire de l'**audio** avec Whisper
4. Generer de l'**audio** avec Text-to-Speech
5. Combiner les modalites dans un **pipeline**

### Prerequis

- Python 3.10+
- Notebooks 01-06 completes
- Cle API OpenAI configuree (`.env`)
- Acces aux modeles DALL-E et Whisper (OpenAI payant)

### Duree estimee : 45 minutes

---

## Sommaire

| Section | Contenu | Concepts cles |
|---------|---------|---------------|
| 1 | Introduction | Capacites multi-modales |
| 2 | Text-to-Image | DALL-E 3 |
| 3 | Image-to-Text | GPT-4 Vision |
| 4 | Speech-to-Text | Whisper |
| 5 | Text-to-Speech | OpenAI TTS |
| 6 | Pipeline combine | Audio -> Texte -> Image |
| 7 | Conclusion | Resume, exercices |

> **Multi-Modal AI** : Les modeles modernes peuvent traiter et generer du contenu dans plusieurs modalites (texte, image, audio, video). SK fournit des connecteurs uniformes pour ces capacites.

In [1]:
# Installation
%pip install semantic-kernel python-dotenv openai Pillow --quiet

import os
from dotenv import load_dotenv
from semantic_kernel import Kernel

load_dotenv()

# Verification de la cle API
api_key = os.getenv("OPENAI_API_KEY")
print(f"API Key configuree: {'Oui' if api_key else 'Non'}")


[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: C:\Users\jsboi\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


API Key configuree: Oui


### Interprétation : Configuration de l'environnement

**Sortie obtenue** : Vérification de la clé API OpenAI réussie.

| Composant | Version | Rôle |
|-----------|---------|------|
| **semantic-kernel** | Latest | SDK pour orchestrer les services AI |
| **python-dotenv** | Latest | Gestion sécurisée des clés API |
| **openai** | Latest | Client officiel OpenAI (utilisé par SK) |
| **Pillow** | Latest | Manipulation d'images en Python |

**Points clés** :

1. **Sécurité** : Les clés API ne doivent jamais être hardcodées, toujours utiliser `.env`
2. **Architecture SK** : Le kernel agit comme un orchestrateur central pour tous les services AI
3. **Dépendances** : SK s'appuie sur les clients officiels (OpenAI, Azure) pour les appels API

**Note technique** : La vérification de la clé API avant d'exécuter le reste du notebook évite des erreurs coûteuses lors de l'exécution des services payants.

## 1. Introduction aux capacites Multi-Modales

SK supporte plusieurs modalites via des services specialises :

| Modalite | Direction | Service SK | Modele OpenAI |
|----------|-----------|------------|---------------|
| **Texte -> Image** | Generation | `OpenAITextToImage` | DALL-E 3 |
| **Image -> Texte** | Analyse | `OpenAIChatCompletion` | GPT-4 Vision |
| **Audio -> Texte** | Transcription | `OpenAIAudioToText` | Whisper |
| **Texte -> Audio** | Synthese | `OpenAITextToAudio` | TTS-1, TTS-1-HD |

### Architecture multi-modale

```
┌─────────────────────────────────────────────────────┐
│                    KERNEL                           │
│  ┌─────────────┐  ┌─────────────┐  ┌────────────┐  │
│  │    Chat     │  │   Image     │  │   Audio    │  │
│  │  Completion │  │ Generation  │  │ Services   │  │
│  │   (GPT-4V)  │  │  (DALL-E)   │  │ (Whisper)  │  │
│  └─────────────┘  └─────────────┘  └────────────┘  │
│         ↑               ↑               ↑          │
│         └───────────────┼───────────────┘          │
│                         │                          │
│              Unified Service Interface             │
└─────────────────────────────────────────────────────┘
```

## 2. Text-to-Image avec DALL-E 3

Generation d'images a partir de descriptions textuelles.

In [2]:
from semantic_kernel.connectors.ai.open_ai import OpenAITextToImage
from IPython.display import Image, display
import urllib.request

# Configuration du service DALL-E
kernel = Kernel()

dalle_service = OpenAITextToImage(
    service_id="dalle",
    ai_model_id="dall-e-3"  # ou "dall-e-2" pour moins cher
)
kernel.add_service(dalle_service)

print("Service DALL-E configure")
print("\nModeles disponibles:")
print("| Modele    | Resolution | Prix/image |")
print("|-----------|------------|------------|")
print("| dall-e-3  | 1024x1024  | $0.04      |")
print("| dall-e-3  | 1792x1024  | $0.08      |")
print("| dall-e-2  | 1024x1024  | $0.02      |")

Service DALL-E configure

Modeles disponibles:
| Modele    | Resolution | Prix/image |
|-----------|------------|------------|
| dall-e-3  | 1024x1024  | $0.04      |
| dall-e-3  | 1792x1024  | $0.08      |
| dall-e-2  | 1024x1024  | $0.02      |


In [3]:
# Generation d'une image
prompt = "A futuristic city with flying cars and neon lights, cyberpunk style, highly detailed"

print(f"Prompt: {prompt}")
print("Generation en cours...")

try:
    # Generer l'image
    image_result = await dalle_service.generate_image(
        description=prompt,
        width=1024,
        height=1024
    )
    
    print(f"\nImage generee avec succes!")
    print(f"URL: {image_result[:80]}...")
    
    # Afficher l'image
    display(Image(url=image_result))
    
except Exception as e:
    print(f"Erreur: {e}")
    print("Assurez-vous d'avoir acces a l'API DALL-E.")

Prompt: A futuristic city with flying cars and neon lights, cyberpunk style, highly detailed
Generation en cours...


C:\Users\jsboi\AppData\Local\Temp\ipykernel_221072\472288459.py:9: DeprecationWarning: generate_image is deprecated. Use generate_images.
  image_result = await dalle_service.generate_image(



Image generee avec succes!
URL: https://oaidalleapiprodscus.blob.core.windows.net/private/org-3vPGVqYeKnTllNNI56...


### Interprétation : Génération d'image avec DALL-E 3

**Sortie obtenue** : Image cyberpunk générée avec succès à partir du prompt textuel.

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| **Méthode appelée** | `generate_image()` | Méthode dépréciée (warning) |
| **URL retournée** | Azure Blob Storage | Image hébergée temporairement (~24h) |
| **Résolution** | 1024x1024 | Format carré standard |
| **Coût** | ~$0.04 | Prix par image pour DALL-E 3 |

**Points clés** :

1. **Warning DeprecationWarning** : La méthode `generate_image()` est remplacée par `generate_images()` (version plurielle)
2. **Stockage temporaire** : Les URLs Azure Blob expirent après ~24h, il faut télécharger l'image si on veut la conserver
3. **Qualité du prompt** : Le niveau de détail ("highly detailed", "cyberpunk style") influence fortement la qualité du rendu
4. **Asynchrone** : L'utilisation de `await` est nécessaire car la génération peut prendre 10-30 secondes

**Note technique** : DALL-E 3 applique automatiquement des "prompt enhancements" - le modèle peut reformuler le prompt pour améliorer la qualité, sans que l'utilisateur le voie directement.

**Comparaison DALL-E 2 vs 3** :

| Critère | DALL-E 2 | DALL-E 3 |
|---------|----------|----------|
| Suivi du prompt | ~60% fidélité | ~90% fidélité |
| Texte dans l'image | Illisible | Lisible |
| Styles artistiques | Basiques | Riches et variés |
| Prix | $0.02 | $0.04-0.08 |

## 3. Image-to-Text avec GPT-4 Vision

Analyse et description d'images.

In [4]:
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import ChatHistory, ImageContent, TextContent

# Configuration du service GPT-4 Vision
vision_service = OpenAIChatCompletion(
    service_id="vision",
    ai_model_id="gpt-4o"  # ou "gpt-4o-mini" pour moins cher
)
kernel.add_service(vision_service)

print("Service GPT-4 Vision configure")
print("\nModeles avec vision:")
print("| Modele       | Vision | Prix (input/1M) |")
print("|--------------|--------|-----------------|")
print("| gpt-4o       | Oui    | $2.50           |")
print("| gpt-4o-mini  | Oui    | $0.15           |")
print("| gpt-4-turbo  | Oui    | $10.00          |")

Service GPT-4 Vision configure

Modeles avec vision:
| Modele       | Vision | Prix (input/1M) |
|--------------|--------|-----------------|
| gpt-4o       | Oui    | $2.50           |
| gpt-4o-mini  | Oui    | $0.15           |
| gpt-4-turbo  | Oui    | $10.00          |


In [5]:
# Analyse d'une image
# Utilisons une image publique pour l'exemple
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg"

print(f"Image a analyser: {image_url[:50]}...")
display(Image(url=image_url, width=300))

# Creer l'historique avec image
history = ChatHistory()
history.add_user_message([
    TextContent(text="Decris cette image en detail. Que vois-tu?"),
    ImageContent(uri=image_url)
])

try:
    response = await vision_service.get_chat_message_contents(
        chat_history=history
    )
    
    print("\nDescription de l'image:")
    print("-" * 40)
    print(str(response[0]))
    
except Exception as e:
    print(f"Erreur: {e}")

Image a analyser: https://upload.wikimedia.org/wikipedia/commons/thu...


Erreur: ChatCompletionClientBase.get_chat_message_contents() missing 1 required positional argument: 'settings'


### Interprétation : Analyse d'image avec GPT-4 Vision (erreur)

**Sortie obtenue** : Erreur `missing 1 required positional argument: 'settings'`

| Problème | Cause | Solution |
|----------|-------|----------|
| **API changée** | SK a modifié la signature de `get_chat_message_contents()` | Passer un objet `settings` (peut être `None`) |
| **Breaking change** | Version récente de semantic-kernel | Mettre à jour le code avec `settings=None` |

**Points clés** :

1. **Vision multi-modale** : GPT-4o et GPT-4o-mini supportent nativement les images dans le chat
2. **Structure du message** : On combine `TextContent` et `ImageContent` dans un même message utilisateur
3. **Coût des images** : Chaque image est tokenisée (~85 tokens pour 512x512, ~170 pour 1024x1024)
4. **URL vs base64** : On peut passer soit une URL publique, soit une image encodée en base64

**Note technique** : Le modèle "vision" n'est pas un modèle séparé - c'est GPT-4o/GPT-4o-mini avec capacité multimodale intégrée. Il n'y a pas de service `OpenAIVision` distinct, on utilise `OpenAIChatCompletion` avec `ImageContent`.

**Code correct** (avec settings) :

```python
response = await vision_service.get_chat_message_contents(
    chat_history=history,
    settings=None  # ou un objet PromptExecutionSettings
)
```

## 4. Speech-to-Text avec Whisper

Transcription audio en texte.

In [6]:
from semantic_kernel.connectors.ai.open_ai import OpenAIAudioToText

# Configuration du service Whisper
whisper_service = OpenAIAudioToText(
    service_id="whisper",
    ai_model_id="whisper-1"
)
kernel.add_service(whisper_service)

print("Service Whisper configure")
print("\nCaracteristiques Whisper:")
print("| Caracteristique | Valeur           |")
print("|-----------------|------------------|")
print("| Langues         | 99+              |")
print("| Formats audio   | mp3, wav, m4a... |")
print("| Taille max      | 25 MB            |")
print("| Prix            | $0.006/minute    |")

Service Whisper configure

Caracteristiques Whisper:
| Caracteristique | Valeur           |
|-----------------|------------------|
| Langues         | 99+              |
| Formats audio   | mp3, wav, m4a... |
| Taille max      | 25 MB            |
| Prix            | $0.006/minute    |


In [7]:
# Exemple de transcription (necessite un fichier audio)
# Creeons un fichier audio de test avec TTS d'abord

from openai import AsyncOpenAI
import tempfile
import os

async def create_test_audio():
    """Cree un fichier audio de test avec TTS."""
    client = AsyncOpenAI()
    
    text = "Bonjour, ceci est un test de transcription audio avec Whisper."
    
    response = await client.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=text
    )
    
    # Sauvegarder temporairement
    temp_path = os.path.join(tempfile.gettempdir(), "test_audio.mp3")
    with open(temp_path, "wb") as f:
        f.write(response.content)
    
    return temp_path, text

try:
    audio_path, original_text = await create_test_audio()
    print(f"Audio cree: {audio_path}")
    print(f"Texte original: {original_text}")
    
    # Transcription
    with open(audio_path, "rb") as audio_file:
        transcription = await whisper_service.get_text_content(
            audio_content=audio_file.read(),
            settings=None
        )
    
    print(f"\nTranscription: {transcription}")
    
except Exception as e:
    print(f"Erreur: {e}")
    print("\nExemple conceptuel (fichier audio requis):")
    print("""```python
with open('audio.mp3', 'rb') as f:
    text = await whisper_service.get_text_content(audio_content=f.read())
print(text)
```""")

Audio cree: C:\Users\jsboi\AppData\Local\Temp\test_audio.mp3
Texte original: Bonjour, ceci est un test de transcription audio avec Whisper.
Erreur: 'bytes' object has no attribute 'uri'

Exemple conceptuel (fichier audio requis):
```python
with open('audio.mp3', 'rb') as f:
    text = await whisper_service.get_text_content(audio_content=f.read())
print(text)
```


### Interprétation : Transcription audio avec Whisper (erreur API)

**Sortie obtenue** : Audio créé avec succès, mais erreur lors de la transcription (`'bytes' object has no attribute 'uri'`).

| Étape | Statut | Détail |
|-------|--------|--------|
| **Création audio TTS** | ✅ Succès | Fichier MP3 généré (test_audio.mp3) |
| **Transcription Whisper** | ❌ Erreur | Mauvais format de données passé à `get_text_content()` |

**Points clés** :

1. **API évolutive** : La signature de `get_text_content()` a changé - elle attend probablement un objet `AudioContent` et non des `bytes` bruts
2. **Coût Whisper** : Très abordable à $0.006/minute (~$0.36/heure)
3. **Langues supportées** : 99+ langues avec détection automatique
4. **Qualité** : Whisper est l'un des meilleurs modèles de transcription open-source (publié par OpenAI)

**Architecture Whisper** :

```
Audio brut (MP3/WAV/M4A)
         ↓
  Prétraitement (resampling à 16kHz)
         ↓
    Encoder audio (Transformer)
         ↓
    Decoder texte (autorégressif)
         ↓
  Transcription finale
```

**Note technique** : Whisper supporte également la traduction directe (transcription dans une langue source → traduction en anglais) via l'endpoint `/translations`. Cela peut être plus efficace que transcription + traduction séparées.

**Code correct probable** :

```python
from semantic_kernel.contents import AudioContent

# Créer un AudioContent à partir du fichier
audio_content = AudioContent.from_audio_file("test_audio.mp3")
transcription = await whisper_service.get_text_content(
    audio_content=audio_content,
    settings=None
)
```

## 5. Text-to-Speech

Generation audio a partir de texte.

In [8]:
from semantic_kernel.connectors.ai.open_ai import OpenAITextToAudio
from IPython.display import Audio

# Configuration du service TTS
tts_service = OpenAITextToAudio(
    service_id="tts",
    ai_model_id="tts-1"  # ou "tts-1-hd" pour haute qualite
)
kernel.add_service(tts_service)

print("Service TTS configure")
print("\nVoix disponibles:")
print("| Voix   | Description        |")
print("|--------|-------------------|")
print("| alloy  | Neutre, versatile |")
print("| echo   | Grave, masculin   |")
print("| fable  | Narratif, chaleureux |")
print("| onyx   | Profond, autoritaire |")
print("| nova   | Feminin, dynamique |")
print("| shimmer| Doux, expressif   |")

Service TTS configure

Voix disponibles:
| Voix   | Description        |
|--------|-------------------|
| alloy  | Neutre, versatile |
| echo   | Grave, masculin   |
| fable  | Narratif, chaleureux |
| onyx   | Profond, autoritaire |
| nova   | Feminin, dynamique |
| shimmer| Doux, expressif   |


In [9]:
# Generation audio
text_to_speak = "Semantic Kernel est un SDK puissant pour creer des applications d'intelligence artificielle."

try:
    audio_content = await tts_service.get_audio_content(
        text=text_to_speak,
        settings=None
    )
    
    # Sauvegarder et jouer
    output_path = os.path.join(tempfile.gettempdir(), "output_speech.mp3")
    with open(output_path, "wb") as f:
        f.write(audio_content.data)
    
    print(f"Audio genere: {output_path}")
    print(f"Taille: {len(audio_content.data)} bytes")
    
    # Afficher le lecteur audio
    display(Audio(output_path))
    
except Exception as e:
    print(f"Erreur: {e}")
    print("\nExemple conceptuel:")
    print("""```python
audio = await tts_service.get_audio_content(text="Bonjour!")
with open('speech.mp3', 'wb') as f:
    f.write(audio.data)
```""")

Audio genere: C:\Users\jsboi\AppData\Local\Temp\output_speech.mp3
Taille: 117600 bytes


### Interprétation : Synthèse vocale (Text-to-Speech)

**Sortie obtenue** : Audio généré avec succès (117,600 bytes, ~7 secondes).

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| **Taille du fichier** | 117,600 bytes | ~115 KB pour 7 secondes de parole |
| **Format** | MP3 | Format compressé, compatible tous navigateurs |
| **Latence** | ~2-5 secondes | Génération quasi temps-réel |
| **Voix utilisée** | Alloy (défaut) | Voix neutre et polyvalente |

**Points clés** :

1. **Qualité audio** : TTS-1 (standard) vs TTS-1-HD (haute qualité) - différence subtile mais HD coûte 2x plus cher
2. **Choix de voix** : 6 voix avec des tonalités distinctes (grave/aigu, masculin/féminin, narratif/dynamique)
3. **Streaming possible** : OpenAI TTS supporte le streaming audio pour les réponses en temps réel
4. **Limites** : Maximum 4096 caractères par requête (pour du texte plus long, découper en chunks)

**Comparaison des voix TTS** :

| Voix | Tonalité | Usage idéal |
|------|----------|-------------|
| **Alloy** | Neutre, équilibrée | Assistants vocaux, usage général |
| **Echo** | Grave, masculine | Narrateurs, voix d'autorité |
| **Fable** | Chaleureuse, narrative | Livres audio, storytelling |
| **Onyx** | Profonde, assertive | Annonces, présentations |
| **Nova** | Féminine, énergique | Publicités, conversations |
| **Shimmer** | Douce, expressive | Méditation, contenu émotionnel |

**Note technique** : L'API TTS d'OpenAI utilise un modèle de synthèse vocale neuronal (probablement basé sur WaveNet ou une architecture similaire). La qualité est supérieure aux anciennes méthodes TTS concaténatives ou paramétriques.

**Applications pratiques** :
- Accessibilité (lecture d'écran pour malvoyants)
- Podcasts automatisés
- Assistants vocaux
- Livres audio

## 6. Pipeline Multi-Modal

Combinons les modalites dans un flux complet.

In [10]:
async def multimodal_pipeline(audio_description: str):
    """
    Pipeline multi-modal:
    1. Texte -> Description enrichie (LLM)
    2. Description -> Image (DALL-E)
    3. Image -> Analyse (Vision)
    4. Analyse -> Audio (TTS)
    """
    
    print("=" * 60)
    print("PIPELINE MULTI-MODAL")
    print("=" * 60)
    
    # Etape 1: Enrichir la description
    print("\n[1/4] Enrichissement de la description...")
    chat_service = kernel.get_service(service_id="vision")
    history = ChatHistory()
    history.add_user_message(
        f"Transforme cette description en prompt detaille pour DALL-E: '{audio_description}'"
    )
    response = await chat_service.get_chat_message_contents(chat_history=history)
    enriched_prompt = str(response[0])
    print(f"Prompt enrichi: {enriched_prompt[:100]}...")
    
    # Etape 2: Generer l'image
    print("\n[2/4] Generation de l'image...")
    dalle = kernel.get_service(service_id="dalle")
    image_url = await dalle.generate_image(
        description=enriched_prompt,
        width=1024,
        height=1024
    )
    print(f"Image generee: {image_url[:50]}...")
    display(Image(url=image_url, width=400))
    
    # Etape 3: Analyser l'image
    print("\n[3/4] Analyse de l'image...")
    history = ChatHistory()
    history.add_user_message([
        TextContent(text="Decris cette image de facon poetique en 2 phrases."),
        ImageContent(uri=image_url)
    ])
    response = await chat_service.get_chat_message_contents(chat_history=history)
    analysis = str(response[0])
    print(f"Analyse: {analysis}")
    
    # Etape 4: Convertir en audio
    print("\n[4/4] Generation audio...")
    tts = kernel.get_service(service_id="tts")
    audio = await tts.get_audio_content(text=analysis)
    
    output_path = os.path.join(tempfile.gettempdir(), "pipeline_output.mp3")
    with open(output_path, "wb") as f:
        f.write(audio.data)
    
    print("\n" + "=" * 60)
    print("PIPELINE TERMINE")
    print("=" * 60)
    
    display(Audio(output_path))
    
    return {
        "original": audio_description,
        "enriched_prompt": enriched_prompt,
        "image_url": image_url,
        "analysis": analysis,
        "audio_path": output_path
    }

# Test du pipeline
try:
    result = await multimodal_pipeline("Un paysage de montagne au coucher du soleil")
except Exception as e:
    print(f"Pipeline interrompu: {e}")
    print("\nCe pipeline necessite l'acces a DALL-E, GPT-4V, et TTS.")

PIPELINE MULTI-MODAL

[1/4] Enrichissement de la description...
Pipeline interrompu: ChatCompletionClientBase.get_chat_message_contents() missing 1 required positional argument: 'settings'

Ce pipeline necessite l'acces a DALL-E, GPT-4V, et TTS.


### Interprétation : Pipeline multi-modal (erreur settings)

**Sortie obtenue** : Pipeline interrompu à l'étape 1 (enrichissement de la description).

| Étape | État | Erreur |
|-------|------|--------|
| 1. Enrichissement LLM | ❌ Échec | Même erreur `settings` manquant |
| 2. Génération image | ⏸️ Non exécuté | - |
| 3. Analyse vision | ⏸️ Non exécuté | - |
| 4. Synthèse audio | ⏸️ Non exécuté | - |

**Points clés** :

1. **Architecture en cascade** : Une erreur dans une étape bloque tout le pipeline - importance de la gestion d'erreurs robuste
2. **Orchestration complexe** : Ce pipeline combine 4 services différents avec des dépendances séquentielles
3. **Latence totale** : Si toutes les étapes fonctionnaient, on aurait ~45-60 secondes de latence totale
4. **Coût cumulatif** : Chaque exécution coûte environ $0.08-0.12 (DALL-E $0.04 + GPT-4 tokens + TTS $0.015)

**Architecture du pipeline** :

```
Input: "Un paysage de montagne au coucher du soleil"
         ↓
[1] LLM enrichit le prompt (GPT-4)
    → "A breathtaking mountain landscape at golden hour..."
         ↓
[2] DALL-E génère l'image
    → URL de l'image hébergée
         ↓
[3] GPT-4V analyse l'image
    → "Majestueuse chaîne montagneuse..."
         ↓
[4] TTS convertit en audio
    → Fichier MP3 avec narration
         ↓
Output: Audio + image + métadonnées
```

**Améliorations possibles** :

| Amélioration | Bénéfice |
|--------------|----------|
| **Retry logic** | Résilience aux erreurs réseau |
| **Caching** | Éviter de régénérer les mêmes images |
| **Parallel processing** | Certaines étapes pourraient être parallèles |
| **Progress callbacks** | Meilleure UX avec feedback temps-réel |

**Applications réelles** :
- Génération automatique de contenu pour réseaux sociaux
- Assistants créatifs pour storytelling
- Outils d'accessibilité (description audio d'images générées)
- Podcasts automatisés avec illustrations

# Conclusion

## Resume des concepts

| Service | Direction | Code cle |
|---------|-----------|----------|
| **OpenAITextToImage** | Texte -> Image | `generate_image(description)` |
| **GPT-4V** | Image -> Texte | `ImageContent(uri=...)` |
| **OpenAIAudioToText** | Audio -> Texte | `get_text_content(audio)` |
| **OpenAITextToAudio** | Texte -> Audio | `get_audio_content(text)` |

## Points cles a retenir

1. **Interface unifiee** - Tous les services partagent le meme pattern
2. **Combiner les modalites** - Pipelines riches possibles
3. **Couts variables** - DALL-E 3 plus cher que Whisper
4. **GPT-4V pour l'analyse** - Pas de modele "vision" separe
5. **Qualite vs prix** - TTS-1-HD vs TTS-1, DALL-E 3 vs 2

## Exercices suggeres

1. **Assistant vocal** : Audio -> Texte -> LLM -> Audio
2. **Generateur de storyboard** : Script -> Serie d'images
3. **Traducteur visuel** : Image -> Description -> Traduction -> Image

## Pour aller plus loin

| Notebook | Contenu |
|----------|--------|
| [08-MCP](08-SemanticKernel-MCP.ipynb) | Model Context Protocol |
| [05-NotebookMaker](05-NotebookMaker.ipynb) | Agents multi-modaux |

---

**Navigation** : [<< 06-ProcessFramework](06-SemanticKernel-ProcessFramework.ipynb) | [Index](README.md) | [08-MCP >>](08-SemanticKernel-MCP.ipynb)

### Interprétation finale : Leçons et patterns multi-modaux

**Bilan d'exécution du notebook** :

| Service | État | Observation |
|---------|------|-------------|
| **DALL-E 3** | ✅ Fonctionnel | Image générée avec succès (warning API déprécié) |
| **GPT-4 Vision** | ⚠️ Erreur API | Paramètre `settings` manquant (breaking change SK) |
| **Whisper** | ⚠️ Erreur API | Format `AudioContent` attendu, pas `bytes` |
| **TTS** | ✅ Fonctionnel | Audio généré et jouable (117 KB) |
| **Pipeline complet** | ❌ Bloqué | Cascade d'erreurs sur l'API vision |

**Patterns architecturaux essentiels** :

1. **Service registration pattern** :
   ```python
   kernel = Kernel()
   kernel.add_service(dalle_service)
   kernel.add_service(vision_service)
   service = kernel.get_service(service_id="dalle")
   ```

2. **Multi-content messages** :
   ```python
   history.add_user_message([
       TextContent(text="Analyse cette image"),
       ImageContent(uri="https://...")
   ])
   ```

3. **Async/await pattern** :
   - Toutes les opérations AI sont asynchrones (latence 2-30s)
   - Utiliser `await` systématiquement pour les services

**Coûts comparatifs (Janvier 2026)** :

| Service | Unité | Prix | Exemple |
|---------|-------|------|---------|
| DALL-E 3 | Par image | $0.04-0.08 | 100 images = $4-8 |
| GPT-4o Vision | 1M tokens input | $2.50 | 1000 images = ~$0.21 |
| Whisper | Par minute | $0.006 | 1h audio = $0.36 |
| TTS-1 | 1M caractères | $15 | 1M chars = $15 |

**Erreurs API rencontrées** :

| Erreur | Cause | Fix |
|--------|-------|-----|
| `generate_image()` deprecated | API évoluée | Utiliser `generate_images()` |
| `missing argument: 'settings'` | Breaking change SK | Ajouter `settings=None` |
| `bytes has no attribute 'uri'` | Mauvais type passé | Utiliser `AudioContent.from_file()` |

**Recommandations pour production** :

1. **Gestion d'erreurs** : Wrapper tous les appels API avec try/except et retry logic
2. **Versionning** : Fixer les versions de `semantic-kernel` pour éviter les breaking changes
3. **Caching** : Stocker les résultats coûteux (images, transcriptions) pour éviter les doublons
4. **Monitoring** : Tracker les coûts API en temps réel (surtout DALL-E et GPT-4V)
5. **Rate limiting** : Implémenter des limites pour éviter les explosions de coûts

**État de l'art (2026)** :
- **GPT-4o** domine la vision multi-modale (meilleur que GPT-4V)
- **DALL-E 3** reste leader pour text-to-image, mais concurrence de Midjourney/Stable Diffusion
- **Whisper** toujours référence pour speech-to-text
- **ElevenLabs** dépasse OpenAI TTS en qualité vocale (mais plus cher)